# Multi-Category ASTERIX Encoding Tutorial

Learn how to encode surveillance data using **all 7 supported ASTERIX categories**:

- **CAT001** - Older Monoradar Target Reports (~7m precision)
- **CAT019** - Multilateration System Status Messages
- **CAT020** - MLAT Target Reports (~0.67m precision)
- **CAT021** - ADS-B Target Reports (~0.021m precision!)
- **CAT034** - Radar Service Messages (North Marker, Sector Crossing)
- **CAT048** - Modern Monoradar Target Reports (~7m precision)
- **CAT062** - System Track Data (~0.67m precision)

This tutorial demonstrates a complete airport surveillance scenario using multiple data sources.

---

## Setup

In [ ]:
import sys
import struct
import time
sys.path.insert(0, '<path-to-asterix-repo>')

from asterix.radar_integration import MockRadar
from asterix.radar_integration.encoder import (
    encode_cat001, encode_cat019_status, encode_cat020,
    encode_cat021, encode_cat034_north_marker, encode_cat048, encode_cat062
)
from asterix.radar_integration.visualization import plot_radar_ascii

print("✅ All 7 ASTERIX category encoders loaded!")

## Scenario: Complete Airport Surveillance

We'll simulate a complete airport surveillance system with:
- Primary radar (CAT001, CAT048)
- MLAT surface tracking (CAT019, CAT020)
- ADS-B transponders (CAT021)
- System tracks (CAT062)
- Service messages (CAT034)

### Step 1: Generate Radar Data

In [ ]:
# Create radar at Berlin Brandenburg Airport
radar = MockRadar(lat=52.5597, lon=13.2877, alt=100.0)
plots = radar.generate_plots(num_targets=10, add_noise=True)

print(f"Generated {len(plots)} radar plots")
print(f"\nSample plot: Range={plots[0].range/1000:.1f} km, Azimuth={plots[0].azimuth:.1f}°")

### Step 2: CAT001 - Older Radar Specification

In [ ]:
cat001_data = encode_cat001(plots, sac=1, sic=1)

print(f"CAT001 (Older Radar):")
print(f"  Category: {cat001_data[0]}")
print(f"  Length: {struct.unpack('!H', cat001_data[1:3])[0]} bytes")
print(f"  {len(plots)} plots → {len(cat001_data)} bytes ({len(cat001_data)/len(plots):.1f} bytes/plot)")
print(f"  Precision: ~7.2 meters (1/128 NM resolution)")

### Step 3: CAT048 - Modern Radar Specification

In [ ]:
cat048_data = encode_cat048(plots, sac=1, sic=2)

print(f"CAT048 (Modern Radar):")
print(f"  {len(plots)} plots → {len(cat048_data)} bytes ({len(cat048_data)/len(plots):.1f} bytes/plot)")
print(f"  Precision: ~7.2 meters (1/256 NM resolution)")
print(f"  Difference from CAT001: {len(cat048_data) - len(cat001_data):+d} bytes ({((len(cat048_data)/len(cat001_data))-1)*100:+.1f}%)")

### Step 4: CAT034 - Radar Service Messages

In [ ]:
# North Marker (antenna passes North)
north_marker = encode_cat034_north_marker(
    sac=1, sic=1,
    sector_number=0,
    antenna_rotation_speed=4.0  # 4-second rotation (15 RPM)
)

print(f"CAT034 (Service Messages):")
print(f"  North Marker: {len(north_marker)} bytes")
print(f"  Rotation: 4.0 seconds (15 RPM)")
print(f"  Use: Antenna synchronization, timing reference")

### Step 5: CAT019 - MLAT System Status

In [ ]:
# MLAT system health status
mlat_status = encode_cat019_status(
    sac=2, sic=1,
    mlat_lat=52.5597,
    mlat_lon=13.2877,
    mlat_height_m=100.0,
    operational=True,
    num_sensors=4
)

print(f"CAT019 (MLAT System Status):")
print(f"  Length: {len(mlat_status)} bytes")
print(f"  Status: Operational")
print(f"  Sensors: 4 active")
print(f"  Use: MLAT health monitoring, coverage status")

### Step 6: CAT020 - MLAT Surface Surveillance

In [ ]:
# Airport surface targets (MLAT)
mlat_targets = [
    {
        'lat': 52.5597, 'lon': 13.2877,
        'aircraft_address': 0x3C6544,
        'callsign': 'DLH123',
        'track_number': 5001,
        'vx': 5.0, 'vy': 3.0,  # Taxiing speed
        'x_m': 1200, 'y_m': 800
    },
    {
        'lat': 52.5601, 'lon': 13.2885,
        'aircraft_address': 0x4009A8,
        'callsign': 'BAW456',
        'track_number': 5002,
        'vx': -3.0, 'vy': 8.0
    }
]

cat020_data = encode_cat020(mlat_targets, sac=2, sic=1)

print(f"CAT020 (MLAT Surface):")
print(f"  {len(mlat_targets)} targets → {len(cat020_data)} bytes ({len(cat020_data)/len(mlat_targets):.1f} bytes/target)")
print(f"  Precision: ~0.67 meters (180/2^25 degrees)")
print(f"  Use: Runway incursion detection, taxiway monitoring")

### Step 7: CAT021 - ADS-B (Highest Precision!)

In [ ]:
# ADS-B reports from equipped aircraft
adsb_reports = [
    {
        'lat': 52.5597, 'lon': 13.2877,
        'aircraft_address': 0x3C6544,
        'callsign': 'DLH123',
        'flight_level': 50,  # FL050 = 5000 ft
        'gnss_height_ft': 5120,
        'airspeed_kt': 200,
        'true_airspeed_kt': 215,
        'magnetic_heading_deg': 350.0,
        'vertical_rate_fpm': -500,
        'track_number': 2001
    }
]

cat021_data = encode_cat021(adsb_reports, sac=3, sic=1)

print(f"CAT021 (ADS-B):")
print(f"  {len(adsb_reports)} reports → {len(cat021_data)} bytes")
print(f"  Precision: ~0.021 meters (180/2^23 degrees)")
print(f"  Improvement: 340x better than radar!")
print(f"  Use: Precision approaches, runway safety")

### Step 8: CAT062 - Fused System Tracks

In [ ]:
# Fused tracks from all sensors
system_tracks = [
    {
        'lat': 52.5597, 'lon': 13.2877,
        'altitude_ft': 5000,
        'vx': 50.0, 'vy': 100.0,  # m/s
        'callsign': 'DLH123',
        'adep': 'EDDM',  # Munich
        'ades': 'EDDB'   # Berlin
    }
]

cat062_data = encode_cat062(system_tracks, sac=4, sic=1)

print(f"CAT062 (Fused Tracks):")
print(f"  {len(system_tracks)} tracks → {len(cat062_data)} bytes")
print(f"  Precision: ~0.67 meters")
print(f"  Includes: Flight plan (departure/destination)")
print(f"  Use: Multi-sensor data fusion")

## Precision Comparison

Let's compare the position encoding precision across categories:

In [ ]:
import pandas as pd

precision_data = {
    'Category': ['CAT001', 'CAT048', 'CAT020', 'CAT062', 'CAT021'],
    'Resolution': ['1/128 NM', '1/256 NM', '180/2^25°', '180/2^25°', '180/2^23°'],
    'Meters': [14.5, 7.2, 0.67, 0.67, 0.021],
    'Use Case': ['Older radar', 'Modern radar', 'MLAT surface', 'Fused tracks', 'ADS-B']
}

df = pd.DataFrame(precision_data)
print("\n📊 Position Precision Comparison:")
print("=" * 80)
print(df.to_string(index=False))
print("=" * 80)
print(f"\n🎯 CAT021 is {7.2/0.021:.0f}x more precise than radar!")

## Complete Surveillance System Summary

In [ ]:
# Calculate total bytes
total_bytes = (
    len(cat001_data) + len(cat019_status) + len(cat020_data) +
    len(cat021_data) + len(north_marker) + len(cat048_data) + len(cat062_data)
)

print("\n" + "=" * 80)
print("Complete Airport Surveillance System")
print("=" * 80)
print(f"\nData generated across all 7 categories:\n")
print(f"  CAT001 (Old Radar):      {len(cat001_data):5d} bytes")
print(f"  CAT019 (MLAT Status):    {len(mlat_status):5d} bytes")
print(f"  CAT020 (MLAT Targets):   {len(cat020_data):5d} bytes")
print(f"  CAT021 (ADS-B):          {len(cat021_data):5d} bytes")
print(f"  CAT034 (Service):        {len(north_marker):5d} bytes")
print(f"  CAT048 (Modern Radar):   {len(cat048_data):5d} bytes")
print(f"  CAT062 (System Tracks):  {len(cat062_data):5d} bytes")
print(f"  {'-' * 40}")
print(f"  TOTAL:                   {total_bytes:5d} bytes")
print(f"\n✅ Complete surveillance data generated!")

## Data Fusion Workflow

This is how the categories work together in a real system:

In [ ]:
workflow = """
┌─────────────┐  CAT001/048 (Radar)
│   Primary   │──────────────────→ Raw plots (~7m accuracy)
│   Radar     │
└─────────────┘
      │
      ├─→ CAT034 (Service) → Timing, synchronization
      │
      ↓
┌─────────────┐  CAT019 (MLAT Status)
│    MLAT     │──────────────────→ System health (4 sensors)
│   System    │  CAT020 (MLAT Targets)
│  (4 sites)  │──────────────────→ Surface position (~0.7m)
└─────────────┘
      │
      ↓
┌─────────────┐  CAT021 (ADS-B)
│   ADS-B     │──────────────────→ High precision (~0.02m!)
│ Transponder │
└─────────────┘
      │
      ↓
┌─────────────┐  CAT062 (System Tracks)
│   Tracker   │──────────────────→ Fused data + flight plan
│  & Fusion   │
└─────────────┘
"""

print(workflow)

## Summary

You've now encoded surveillance data using **all 7 ASTERIX categories**!

### Key Takeaways:

1. **Different categories for different purposes**
   - Radar: CAT001, CAT048
   - MLAT: CAT019 (status), CAT020 (targets)
   - ADS-B: CAT021
   - Fusion: CAT062
   - Service: CAT034

2. **Precision varies 340x!**
   - CAT048: 7.2m (radar)
   - CAT062: 0.67m (fused)
   - CAT021: 0.021m (sub-meter!)

3. **Complete surveillance coverage**
   - Air surveillance (radar)
   - Surface tracking (MLAT)
   - Transponder data (ADS-B)
   - System health (status messages)

### Next Steps:

- Try `03_Flight_Simulator_Integration.ipynb` for ArduPilot/JSBSim
- Try `04_Advanced_Visualization.ipynb` for matplotlib plots
- Run the CLI: `python3 -m asterix.radar_integration.cli benchmark`

**You now have complete ASTERIX encoding mastery!** 🎓